In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import (train_test_split,
                                    KFold,
                                    ParameterGrid,
                                    cross_val_score,
                                    GridSearchCV,
                                    RandomizedSearchCV)
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import (StandardScaler,
                                   OneHotEncoder)
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import make_pipeline, Pipeline
from category_encoders import WOEEncoder, SumEncoder
from tqdm import tqdm_notebook
from datasets import load_dataset
from tqdm import tqdm
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import GridSearchCV

import catboost
import xgboost
from catboost import CatBoostClassifier
from catboost import CatBoostRegressor
from xgboost import XGBClassifier
import nbformat
import matplotlib.pyplot as plt
import seaborn as sns
import itertools

In [4]:
ds = pd.read_csv("datasetxlnocat.csv")
ds = pd.DataFrame(ds)
ds

,Project,Type of project,Revenue,commercial costs,indirect costs,Oibda
0,1,1,1000000,100000,50000,850000
1,2,2,1300000,110000,55000,1135000
2,3,3,1600000,120000,60000,1420000
3,4,1,1900000,130000,65000,1705000
4,5,2,2200000,140000,70000,1990000
5,6,3,2500000,150000,75000,2275000
6,7,1,2800000,160000,80000,2560000
7,8,2,3100000,170000,85000,2845000
8,9,3,3400000,180000,90000,3130000
9,10,1,3700000,190000,95000,3415000


In [5]:
# Определение целевого параметра для обучения
X = ds.drop(['Oibda'], axis=1)
y = ds['Oibda']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
cb_params = {'verbose': False,
             'boost_from_average': True,
             'loss_function': 'MAE'
            }

In [7]:
#Обучение
param_grid_catboost = {'learning_rate': [0.03, 0.1],
                       'depth': [4, 6, 10],
                       'l2_leaf_reg': [1, 3, 5, 7, 9]}
Modelcb = CatBoostRegressor(**cb_params)
Modelcb.grid_search(param_grid_catboost, 
                                       X=X_train, 
                                       y=y_train,
                                       plot=False)



bestTest = 358405.0344
bestIteration = 617

0:	loss: 358405.0343608	best: 358405.0343608 (0)	total: 220ms	remaining: 6.39s

bestTest = 289315.288
bestIteration = 231

1:	loss: 289315.2880398	best: 289315.2880398 (1)	total: 344ms	remaining: 4.81s

bestTest = 288465.3771
bestIteration = 76

2:	loss: 288465.3770512	best: 288465.3770512 (2)	total: 519ms	remaining: 4.67s

bestTest = 391706.5191
bestIteration = 271

3:	loss: 391706.5191089	best: 288465.3770512 (2)	total: 665ms	remaining: 4.32s

bestTest = 308268.0402
bestIteration = 70

4:	loss: 308268.0402126	best: 288465.3770512 (2)	total: 880ms	remaining: 4.4s

bestTest = 399368.2929
bestIteration = 272

5:	loss: 399368.2929174	best: 288465.3770512 (2)	total: 1s	remaining: 4.01s

bestTest = 321844.699
bestIteration = 63

6:	loss: 321844.6990369	best: 288465.3770512 (2)	total: 1.13s	remaining: 3.71s

bestTest = 433335.0388
bestIteration = 264

7:	loss: 433335.0387525	best: 288465.3770512 (2)	total: 1.26s	remaining: 3.46s

bestTest = 21303

{'params': {'depth': 10, 'learning_rate': 0.1, 'l2_leaf_reg': 5},
 'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
               44,
               45,
               46,
    

In [8]:
# Параметры модели
display(Modelcb.best_score_)
display(Modelcb.is_fitted())
display(Modelcb.get_params())
display(Modelcb.get_all_params())
display(Modelcb.score(X_test, y_test))

{'learn': {'MAE': 1.575889997184302e-09}}

True

{'loss_function': 'MAE',
 'verbose': False,
 'boost_from_average': True,
 'depth': 10,
 'learning_rate': 0.1,
 'l2_leaf_reg': 5}

{'nan_mode': 'Min',
 'eval_metric': 'MAE',
 'iterations': 1000,
 'sampling_frequency': 'PerTree',
 'leaf_estimation_method': 'Exact',
 'random_score_type': 'NormalWithModelSizeDecrease',
 'grow_policy': 'SymmetricTree',
 'penalties_coefficient': 1,
 'boosting_type': 'Plain',
 'model_shrink_mode': 'Constant',
 'feature_border_type': 'GreedyLogSum',
 'bayesian_matrix_reg': 0.10000000149011612,
 'eval_fraction': 0,
 'force_unit_auto_pair_weights': False,
 'l2_leaf_reg': 5,
 'random_strength': 1,
 'rsm': 1,
 'boost_from_average': True,
 'model_size_reg': 0.5,
 'pool_metainfo_options': {'tags': {}},
 'subsample': 1,
 'use_best_model': False,
 'random_seed': 0,
 'depth': 10,
 'posterior_sampling': False,
 'border_count': 254,
 'classes_count': 0,
 'auto_class_weights': 'None',
 'sparse_features_conflict_fraction': 0,
 'leaf_estimation_backtracking': 'AnyImprovement',
 'best_model_min_trees': 1,
 'model_shrink_rate': 0,
 'min_data_in_leaf': 1,
 'loss_function': 'MAE',
 'learning_rate': 0.1000

0.9266739802974567

In [9]:
ds.iloc[10]

Project                  11
Type of project           2
Revenue             4000000
commercial costs     200000
indirect costs       100000
Oibda               3700000
Name: 10, dtype: int64

In [10]:
#Загрузка параметров:
data = {'Project': [11], 'Type of project': [2], 'Revenue': [4000000], 'commercial costs': [200000], 'indirect costs': [100000]}
data = pd.DataFrame(data)

In [11]:
data

,Project,Type of project,Revenue,commercial costs,indirect costs
0,11,2,4000000,200000,100000


In [12]:
Modelcb.predict(data)

array([3699999.99999884])

In [17]:
import sweetviz as sv
report = sv.analyze([ds, "DS"])

Done! Use 'show' commands to display/save.   |██████████| [100%]   00:00 -> (00:00 left)


In [ ]:
report.show_html('common analysis.html')